In [4]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re # para mexer com regexp
from keras.utils import to_categorical

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1° Modelo - Somente 1 livro

In [1]:
import requests

#Murray Leinster
#https://pt.wikipedia.org/wiki/Murray_Leinster

texto = requests.get('https://www.gutenberg.org/cache/epub/70392/pg70392.txt').text

In [2]:
print(texto)

﻿The Project Gutenberg eBook of Nobody saw the ship, by Murray
Leinster

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: Nobody saw the ship

Author: Murray Leinster

Illustrator: C. A. Murphy

Release Date: March 27, 2023 [eBook #70392]

Language: English

Produced by: Greg Weeks, Mary Meehan and the Online Distributed
             Proofreading Team at http://www.pgdp.net

*** START OF THE PROJECT GUTENBERG EBOOK NOBODY SAW THE SHIP ***






                          Nobody saw the Ship

                          A POWERFUL NOVELET

                          by Murray Leinster

          

In [6]:
cleaned = re.sub(r'\W+', ' ', texto).lower()
tokens = word_tokenize(cleaned)

train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [7]:
vocabulary_size

2304

In [8]:
len(n_sequences)

10395

In [9]:
train_inputs = n_sequences[:, :-1]
train_targets = n_sequences[:, -1]
train_targets = to_categorical(train_targets, num_classes = vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_inputs.shape)
print(train_targets[0])

(10395, 3)
[0. 0. 0. ... 0. 0. 0.]


In [10]:
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              6912      
                                                                 
 lstm (LSTM)                 (None, 3, 50)             10800     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 2304)              117504    
                                                                 
Total params: 157,966
Trainable params: 157,966
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
model.fit(train_inputs, train_targets, epochs=100, verbose=1)

Epoch 1/100
325/325 [==============================] - 2s 7ms/step - loss: 2.3405 - accuracy: 0.4023
Epoch 2/100
325/325 [==============================] - 3s 8ms/step - loss: 2.3200 - accuracy: 0.4019
Epoch 3/100
325/325 [==============================] - 2s 7ms/step - loss: 2.3254 - accuracy: 0.4048
Epoch 4/100
325/325 [==============================] - 2s 7ms/step - loss: 2.3055 - accuracy: 0.4075
Epoch 5/100
325/325 [==============================] - 2s 7ms/step - loss: 2.2999 - accuracy: 0.4075
Epoch 6/100
325/325 [==============================] - 2s 7ms/step - loss: 2.2928 - accuracy: 0.4036
Epoch 7/100
325/325 [==============================] - 3s 9ms/step - loss: 2.2769 - accuracy: 0.4107
Epoch 8/100
325/325 [==============================] - 2s 7ms/step - loss: 2.2773 - accuracy: 0.4142
Epoch 9/100
325/325 [==============================] - 3s 8ms/step - loss: 2.2612 - accuracy: 0.4209
Epoch 10/100
325/325 [==============================] - 2s 7ms/step - loss: 2.2457 - accura

In [53]:
#from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

def input_autocomplete_text(model, seq_len, text):

    encoded_text = tokenizer.texts_to_sequences([text])[0]
    pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
    print("Input Text:", text)

    for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
        pred_word = tokenizer.index_word[i]
        print("Next word suggestion:", pred_word)

def input_text_strip():
    input_text = input().strip().lower()
    return input_text



# 1° Modelo - Predict do modelo

In [32]:
input_autocomplete_text(model=model, seq_len=seq_len, text = input_text_strip())

He was a
1/1 [==============================] - 0s 18ms/step
Next word suggestion: highly
Next word suggestion: part
Next word suggestion: danger


In [35]:
input_autocomplete_text(model=model, seq_len=seq_len, text = input_text_strip())

She was a
1/1 [==============================] - 0s 18ms/step
Next word suggestion: magnificent
Next word suggestion: coyote
Next word suggestion: most


In [36]:
input_autocomplete_text(model=model, seq_len=seq_len, text = input_text_strip())

He is a
1/1 [==============================] - 0s 20ms/step
Next word suggestion: part
Next word suggestion: creature
Next word suggestion: cross


In [37]:
input_autocomplete_text(model=model, seq_len=seq_len, text = input_text_strip())

She is a
1/1 [==============================] - 0s 18ms/step
Next word suggestion: buzzard
Next word suggestion: modification
Next word suggestion: machine


In [39]:
input_autocomplete_text(model=model, seq_len=seq_len, text = input_text_strip())

he is the
1/1 [==============================] - 0s 21ms/step
Next word suggestion: switch
Next word suggestion: part
Next word suggestion: looked


In [41]:
input_autocomplete_text(model=model, seq_len=seq_len, text = input_text_strip())

She is the
1/1 [==============================] - 0s 19ms/step
Next word suggestion: address
Next word suggestion: faint
Next word suggestion: united


# 1° Modelo - Considerações.
1- Como Murray Leinster foi um escritor de ficção cientistica e em algumas de suas historias a mulher retratada é um robo, podemos ver que isso fica bem retratado nos predicts, como: "she is a machine" ou "she is a modification". Enquanto a mulher é retratada como um robo, o homem é tratado como algo perigoso, como "He is a danger".

2- Alguns predicts estão meio que incompletados se usamos somente o predict da quarta palavra, como: "he is a part" ou "he is a cross".

# 2° Modelo - 5 Livros.

In [43]:
import requests

#Murray Leinster
#https://pt.wikipedia.org/wiki/Murray_Leinster

texto = requests.get('https://www.gutenberg.org/cache/epub/70392/pg70392.txt').text
texto = texto + ' ' + requests.get('https://www.gutenberg.org/cache/epub/70562/pg70562.txt').text
texto = texto + ' ' + requests.get('https://www.gutenberg.org/cache/epub/70638/pg70638.txt').text
texto = texto + ' ' + requests.get('https://www.gutenberg.org/cache/epub/70610/pg70610.txt').text
texto = texto + ' ' + requests.get('https://www.gutenberg.org/cache/epub/70277/pg70277.txt').text

In [45]:
print(texto)

﻿The Project Gutenberg eBook of Nobody saw the ship, by Murray
Leinster

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: Nobody saw the ship

Author: Murray Leinster

Illustrator: C. A. Murphy

Release Date: March 27, 2023 [eBook #70392]

Language: English

Produced by: Greg Weeks, Mary Meehan and the Online Distributed
             Proofreading Team at http://www.pgdp.net

*** START OF THE PROJECT GUTENBERG EBOOK NOBODY SAW THE SHIP ***






                          Nobody saw the Ship

                          A POWERFUL NOVELET

                          by Murray Leinster

          

In [44]:
cleaned = re.sub(r'\W+', ' ', texto).lower()
tokens = word_tokenize(cleaned)

train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [46]:
vocabulary_size

8134

In [47]:
len(n_sequences)

94522

In [48]:
train_inputs = n_sequences[:, :-1]
train_targets = n_sequences[:, -1]
train_targets = to_categorical(train_targets, num_classes = vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_inputs.shape)
print(train_targets[0])

(94522, 3)
[0. 0. 0. ... 0. 0. 0.]


In [49]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 3)              24402     
                                                                 
 lstm_2 (LSTM)               (None, 3, 50)             10800     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 8134)              414834    
                                                                 
Total params: 472,786
Trainable params: 472,786
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
model.fit(train_inputs, train_targets, epochs=100, verbose=1)

Epoch 1/100
2954/2954 [==============================] - 39s 12ms/step - loss: 6.9062 - accuracy: 0.0553
Epoch 2/100
2954/2954 [==============================] - 26s 9ms/step - loss: 6.5152 - accuracy: 0.0628
Epoch 3/100
2954/2954 [==============================] - 27s 9ms/step - loss: 6.2134 - accuracy: 0.0842
Epoch 4/100
2954/2954 [==============================] - 29s 10ms/step - loss: 5.9449 - accuracy: 0.0968
Epoch 5/100
2954/2954 [==============================] - 27s 9ms/step - loss: 5.7456 - accuracy: 0.1059
Epoch 6/100
2954/2954 [==============================] - 24s 8ms/step - loss: 5.5620 - accuracy: 0.1135
Epoch 7/100
2954/2954 [==============================] - 24s 8ms/step - loss: 5.3916 - accuracy: 0.1215
Epoch 8/100
2954/2954 [==============================] - 27s 9ms/step - loss: 5.2399 - accuracy: 0.1290
Epoch 9/100
2954/2954 [==============================] - 25s 8ms/step - loss: 5.1077 - accuracy: 0.1375
Epoch 10/100
2954/2954 [==============================] - 25s 

# 2° Modelo - Predict do modelo

In [54]:
list_text_to_predict = list()
list_text_to_predict.append('he was a')
list_text_to_predict.append('she was a')
list_text_to_predict.append('he is a')
list_text_to_predict.append('she is a')
list_text_to_predict.append('he is the')
list_text_to_predict.append('she is the')

In [55]:
for text in list_text_to_predict:
    input_autocomplete_text(model=model, seq_len=seq_len, text=text)

Input Text: he was a
1/1 [==============================] - 1s 603ms/step
Next word suggestion: mistake
Next word suggestion: phoney
Next word suggestion: ribbon
Input Text: she was a
1/1 [==============================] - 0s 19ms/step
Next word suggestion: phoney
Next word suggestion: mistake
Next word suggestion: good
Input Text: he is a
1/1 [==============================] - 0s 18ms/step
Next word suggestion: long
Next word suggestion: phoney
Next word suggestion: med
Input Text: she is a
1/1 [==============================] - 0s 18ms/step
Next word suggestion: long
Next word suggestion: med
Next word suggestion: good
Input Text: he is the
1/1 [==============================] - 0s 20ms/step
Next word suggestion: devil
Next word suggestion: strongest
Next word suggestion: gaunt
Input Text: she is the
1/1 [==============================] - 0s 22ms/step
Next word suggestion: devil
Next word suggestion: gaunt
Next word suggestion: oldest


# 2° Modelo - Considerações.
1 - Com 5 livros a loss ficou maior e a acuracia ficou substanciamente menor cerca de 20% a menos, o que pode interferir diretamente nas predições.
Tendo isso em mente, vamos aos resultados.

2 - A predição ficou de forma geral muito parecida tanto para homem quanto para mulher. 

3 - Vemos uma mudança em como tanto o homem quanto a mulher é tratada em relação ao modelo 1, pois foram adicionados mais livros com temas diferentes, o que mostra que o autor nao costuma tratar sempre a mulher como "machine" e o homem como "danger", ou seja, tanto o homem quanto a mulher nao sao tratados da mesma forma em todas as historias. 


# 3° Modelo - Alterando para menos a arquitetura da rede LSTM

In [56]:
import requests

#Murray Leinster
#https://pt.wikipedia.org/wiki/Murray_Leinster

texto = requests.get('https://www.gutenberg.org/cache/epub/70392/pg70392.txt').text

In [57]:
cleaned = re.sub(r'\W+', ' ', texto).lower()
tokens = word_tokenize(cleaned)

train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [58]:
train_inputs = n_sequences[:, :-1]
train_targets = n_sequences[:, -1]
train_targets = to_categorical(train_targets, num_classes = vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_inputs.shape)
print(train_targets[0])

(10395, 3)
[0. 0. 0. ... 0. 0. 0.]


In [63]:
model_menos = Sequential()
model_menos.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model_menos.add(LSTM(50,return_sequences=True))
model_menos.add(LSTM(50))
model_menos.add(Dense(vocabulary_size, activation='softmax'))
print(model_menos.summary())
# compile network
model_menos.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 3, 3)              6912      
                                                                 
 lstm_6 (LSTM)               (None, 3, 50)             10800     
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dense_7 (Dense)             (None, 2304)              117504    
                                                                 
Total params: 155,416
Trainable params: 155,416
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
model_menos.fit(train_inputs, train_targets, epochs=100, verbose=1)

Epoch 1/100
325/325 [==============================] - 9s 17ms/step - loss: 6.7375 - accuracy: 0.0620
Epoch 2/100
325/325 [==============================] - 2s 7ms/step - loss: 6.2913 - accuracy: 0.0715
Epoch 3/100
325/325 [==============================] - 2s 7ms/step - loss: 6.2414 - accuracy: 0.0715
Epoch 4/100
325/325 [==============================] - 2s 7ms/step - loss: 6.2040 - accuracy: 0.0715
Epoch 5/100
325/325 [==============================] - 2s 6ms/step - loss: 6.1810 - accuracy: 0.0715
Epoch 6/100
325/325 [==============================] - 3s 9ms/step - loss: 6.1620 - accuracy: 0.0715
Epoch 7/100
325/325 [==============================] - 3s 9ms/step - loss: 6.1437 - accuracy: 0.0715
Epoch 8/100
325/325 [==============================] - 2s 7ms/step - loss: 6.1283 - accuracy: 0.0715
Epoch 9/100
325/325 [==============================] - 2s 7ms/step - loss: 6.1097 - accuracy: 0.0716
Epoch 10/100
325/325 [==============================] - 2s 7ms/step - loss: 6.0908 - accur

# 3° Modelo - Predict.

In [67]:
for text in list_text_to_predict:
    input_autocomplete_text(model=model_menos, seq_len=seq_len, text=text)

Input Text: he was a
1/1 [==============================] - 1s 615ms/step
Next word suggestion: highly
Next word suggestion: fond
Next word suggestion: thousand
Input Text: she was a
1/1 [==============================] - 0s 22ms/step
Next word suggestion: cross
Next word suggestion: exception
Next word suggestion: other
Input Text: he is a
1/1 [==============================] - 0s 19ms/step
Next word suggestion: flea
Next word suggestion: curious
Next word suggestion: twenty
Input Text: she is a
1/1 [==============================] - 0s 19ms/step
Next word suggestion: non
Next word suggestion: edge
Next word suggestion: directly
Input Text: he is the
1/1 [==============================] - 0s 18ms/step
Next word suggestion: trigger
Next word suggestion: local
Next word suggestion: deer
Input Text: she is the
1/1 [==============================] - 0s 22ms/step
Next word suggestion: address
Next word suggestion: raider
Next word suggestion: united


# 3° Modelo - Considerações.
1- A loss/acc ficou maior do que a do primeiro modelo, talvez seja pelo fato dos pesos e vieses aleatorios inicializados pela rede terem sido melhores para o modelo 1.

2- Com a diminuição da rede, tivemos uma mudança substancial nas predições. Comparando com o modelo 1, nao da para dizer como exatamente ele ve o homem e a mulher em sua historia. 

# 4° Modelo - Alterando para mais a arquitetura da rede LSTM. Coloquei mais layers, funções de ativação e regularizações.

In [91]:
from keras.layers.serialization import regularization
model_mais = Sequential()
model_mais.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model_mais.add(LSTM(50,return_sequences=True))
model_mais.add(LSTM(150, activation='tanh', recurrent_activation='sigmoid', kernel_regularizer='l1'))
model_mais.add(Dense(200, activation='relu', kernel_regularizer='l1'))
model_mais.add(Dense(50, activation='relu', kernel_regularizer='l1'))
model_mais.add(Dense(vocabulary_size, activation='softmax'))
print(model_mais.summary())
# compile network
model_mais.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 3, 3)              6912      
                                                                 
 lstm_53 (LSTM)              (None, 3, 50)             10800     
                                                                 
 lstm_54 (LSTM)              (None, 150)               120600    
                                                                 
 dense_29 (Dense)            (None, 200)               30200     
                                                                 
 dense_30 (Dense)            (None, 50)                10050     
                                                                 
 dense_31 (Dense)            (None, 2304)              117504    
                                                                 
Total params: 296,066
Trainable params: 296,066
Non-t

In [92]:
model_mais.fit(train_inputs, train_targets, epochs=100, verbose=1)

Epoch 1/100
325/325 [==============================] - 11s 18ms/step - loss: 12.4086 - accuracy: 0.0694
Epoch 2/100
325/325 [==============================] - 3s 8ms/step - loss: 6.4314 - accuracy: 0.0715
Epoch 3/100
325/325 [==============================] - 3s 9ms/step - loss: 6.3926 - accuracy: 0.0715
Epoch 4/100
325/325 [==============================] - 2s 8ms/step - loss: 6.3753 - accuracy: 0.0715
Epoch 5/100
325/325 [==============================] - 2s 7ms/step - loss: 6.3599 - accuracy: 0.0715
Epoch 6/100
325/325 [==============================] - 2s 7ms/step - loss: 6.3424 - accuracy: 0.0715
Epoch 7/100
325/325 [==============================] - 3s 8ms/step - loss: 6.3320 - accuracy: 0.0715
Epoch 8/100
325/325 [==============================] - 3s 8ms/step - loss: 6.3300 - accuracy: 0.0715
Epoch 9/100
325/325 [==============================] - 2s 7ms/step - loss: 6.3298 - accuracy: 0.0715
Epoch 10/100
325/325 [==============================] - 3s 8ms/step - loss: 6.3302 - acc

# 4° Modelo - Predict.

In [93]:
for text in list_text_to_predict:
    input_autocomplete_text(model=model_mais, seq_len=seq_len, text=text)

Input Text: he was a
1/1 [==============================] - 1s 623ms/step
Next word suggestion: the
Next word suggestion: of
Next word suggestion: and
Input Text: she was a
1/1 [==============================] - 0s 21ms/step
Next word suggestion: the
Next word suggestion: of
Next word suggestion: and
Input Text: he is a
1/1 [==============================] - 0s 20ms/step
Next word suggestion: the
Next word suggestion: of
Next word suggestion: and
Input Text: she is a
1/1 [==============================] - 0s 18ms/step
Next word suggestion: the
Next word suggestion: of
Next word suggestion: and
Input Text: he is the
1/1 [==============================] - 0s 20ms/step
Next word suggestion: the
Next word suggestion: of
Next word suggestion: and
Input Text: she is the
1/1 [==============================] - 0s 21ms/step
Next word suggestion: the
Next word suggestion: of
Next word suggestion: and


# 4° Modelo - Considerações.
1 - Com mais layers, funções de ativação e regularizações, a loss e acuracia ficaram as piores em relação aos outros modelos, mostrando que nem sempre uma arquitetura mais "deep" é a melhor. Também fica claro como os resultados são extremamente dependentes de uma boa arquitetura.

2 - Como vimos, a loss/acuracia como resultados ruins, da claramente para perceber que as predições dessa rede também foram ruins. Totalmente inconclusivos.

# Considerações Finais
1- Em minha opinião, o modelo 1 teve os melhores resultados visiveis, alem da melhor loss e acurácia, o que refletiu claramente nas predições.

2- Com os testes feitos, deu para concluir que arquiteturas mais complexas não funcionam bem para modelos textuais.

Qualquer duvida, ficou a disposição
Obrigado.

Kaio.
